## **Algorithmic Question (AQ):**

*Arya needs to travel between cities using a network of flights. Each flight has a fixed cost (in euros), and she wants to find the cheapest possible way to travel from her starting city to her destination city. However, there are some constraints on the journey:*

- Arya can make at most k stops during her trip (this means up to k+1 flights).
    
- If no valid route exists within these constraints, the result should be -1.

**Given a graph of cities connected by flights, your job is to find the minimum cost for Arya to travel between two specified cities (src to dst) while following the constraints.**



#### **Your Task**
- a) *Write a pseudocode that describes the algorithm to find the cheapest route with at most k stops.*

- b) *Implement the algorithm in Python and simulate the given test cases.*

- c) *Analyze the algorithm's efficiency. Provide its time complexity and space complexity, and explain whether it is efficient for large graphs (e.g., n > 100).*

- d) *Optimize the algorithm to handle larger graphs. Provide an updated pseudocode and analyze the computational complexity of your optimization.*

- e) *Ask LLM (e.g., ChatGPT) for an optimized version of your algorithm. Compare its solution to yours in terms of performance, time complexity, and correctness.*



### a) *Write a pseudocode that describes the algorithm to find the cheapest route with at most k stops.*

## Pseudocode for the Basic Algorithm FindCheapestRoute(n, flights, src, dst, k)

    // Create adjacency list representation of flights
    INITIALIZE graph as a dictionary of lists
    FOR EACH flight IN flights DO
        ADD (destination, cost) to graph[source]
    
    // Queue to manage routes
    INITIALIZE queue as a deque
    queue.push((src, 0, k+1))  // (current_city, total_cost, stops_remaining)
    
    // Visited state tracking
    INITIALIZE visited as a dictionary with default value as infinity
    visited[(src, k+1)] = 0  // Minimum cost to reach src with k+1 stops is 0
    
    // Start exploration
    WHILE queue is not empty DO
        (current_city, current_cost, stops_left) = queue.popLeft()
        
        // Destination reached
        IF current_city == dst THEN
            RETURN current_cost
        
        // Explore neighboring cities if stops are available
        IF stops_left > 0 THEN
            FOR EACH (neighbor, flight_cost) IN graph[current_city] DO
                new_cost = current_cost + flight_cost
                
                // Update only if the new cost is better
                IF new_cost < visited[(neighbor, stops_left - 1)] THEN
                    visited[(neighbor, stops_left - 1)] = new_cost
                    queue.push((neighbor, new_cost, stops_left - 1))
    
    // No route found
    RETURN -1

In [45]:
from heapq import heappush, heappop

def find_cheapest_route_v1(n, flights, src, dst, k):
    # create adjacence list 
    graph = [[] for _ in range(n)]
    for start, end, price in flights :
        graph[start].append((end, price))

    # priority queue to store (total_cost , current_city , stops_remaining)
    # we use min heap to always process cheapest routes first
    heap = [(0, src, k+1)]

    min_cost = {}

    while heap:
        current_cost, current_city, remaining_stops = heappop(heap)

        if current_city == dst :
            return current_cost
        
        if remaining_stops <= 0 :
            continue 

        for next_city , flight_cost in graph[current_city]:
            new_cost = current_cost + flight_cost
            new_stops = remaining_stops - 1

            key = (next_city, new_stops)
            if ( key not in min_cost or new_cost < min_cost[key]):
                min_cost[key] = new_cost
                heappush(heap, (new_cost, next_city, new_stops))
    return -1


#### *Another version :*

In [46]:
from heapq import heappush, heappop

def find_cheapest_route_v2(n, flights, src, dst, k):
    # Create adjacency list
    graph = [[] for _ in range(n)]
    for start, end, price in flights:
        graph[start].append((end, price))
    
    # Priority queue to store (total_cost, current_city, stops_remaining)
    # We use min heap to always process cheapest routes first
    heap = [(0, src, k+1)]
    
    # Keep track of minimum costs
    # Key: (city, remaining_stops)
    # Value: minimum cost to reach that city with those remaining stops
    min_costs = {}
    
    while heap:
        current_cost, current_city, remaining_stops = heappop(heap)
        
        # Reached destination
        if current_city == dst:
            return current_cost
        
        # No more stops allowed
        if remaining_stops <= 0:
            continue
        
        # Explore neighboring cities
        for next_city, flight_cost in graph[current_city]:
            new_cost = current_cost + flight_cost
            new_stops = remaining_stops - 1
            
            # Key optimization: 
            # Only add to heap if:
            # 1. We haven't seen this (city, stops) combination before
            # 2. Or we've found a cheaper route to this city with same/more stops
            key = (next_city, new_stops)
            if (key not in min_costs or new_cost < min_costs[key]):
                min_costs[key] = new_cost
                heappush(heap, (new_cost, next_city, new_stops))
    
    # No route found
    return -1

##### *Tests:* 

In [47]:
n = 4  
flights = [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]]  
src = 0  
dst = 3  
k = 1  

###

print(find_cheapest_route_v1(n, flights, src, dst, k))
print(find_cheapest_route_v2(n, flights, src, dst, k))

700
700


In [48]:
n = 3  
flights = [[0, 1, 100], [1, 2, 100], [0, 2, 500]]  
src = 0  
dst = 2  
k = 1  
find_cheapest_route_v1(n, flights, src, dst, k)

200

In [49]:
n = 3  
flights = [[0, 1, 100], [1, 2, 100], [0, 2, 500]]  
src = 0  
dst = 2  
k = 0  

find_cheapest_route_v1(n, flights, src, dst, k)

500

In [50]:
n = 4  
flights = [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]]  
src = 0  
dst = 3  
k = 2  
find_cheapest_route_v1(n, flights, src, dst, k)

400

In [51]:
n = 4  
flights = [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]]  
src = 0  
dst = 3  
k = 2  
find_cheapest_route_v1(n, flights, src, dst, k)


400

#### **Tests  with larger data**

##### *I've decited to make a funciton ton generate random syntetic data*

In [52]:
import random
import time

def generate_synthetic_data(num_cities, num_flights, max_cost):
    """
    Generate a synthetic dataset for testing the algorithm.
    
    Args:
        num_cities (int): Number of cities (nodes in the graph).
        num_flights (int): Number of flights (edges in the graph).
        max_cost (int): Maximum cost of a flight.
    
    Returns:
        tuple: (n, flights, src, dst, k)
    """
    flights = []
    for _ in range(num_flights):
        start = random.randint(0, num_cities - 1)
        end = random.randint(0, num_cities - 1)
        while end == start:  # Ensure no self-loops
            end = random.randint(0, num_cities - 1)
        cost = random.randint(1, max_cost)
        flights.append([start, end, cost])
    
    src = random.randint(0, num_cities - 1)
    dst = random.randint(0, num_cities - 1)
    while dst == src:  # Ensure source and destination are different
        dst = random.randint(0, num_cities - 1)
    
    k = random.randint(0, num_cities - 1)  # Randomize max stops
    
    return num_cities, flights, src, dst, k


def test_algorithm_with_synthetic_data(algorithm, num_tests=5, num_cities=100, num_flights=1000, max_cost=500):
    """
    Test the algorithm on synthetic datasets and measure performance.
    
    Args:
        algorithm (function): The algorithm function to test.
        num_tests (int): Number of test runs.
        num_cities (int): Number of cities for each test.
        num_flights (int): Number of flights for each test.
        max_cost (int): Maximum flight cost.
    
    Returns:
        None
    """
    for i in range(num_tests):
        print(f"Test {i+1}:")
        n, flights, src, dst, k = generate_synthetic_data(num_cities, num_flights, max_cost)
        start_time = time.time()
        result = algorithm(n, flights, src, dst, k)
        end_time = time.time()
        print(f"Source: {src}, Destination: {dst}, Max Stops: {k}")
        print(f"Result: {result}")
        print(f"Time Taken: {end_time - start_time:.4f} seconds")
        print("-" * 40)

In [53]:
# Testare l'algoritmo su dataset sintetici
test_algorithm_with_synthetic_data(find_cheapest_route_v1, num_tests=5, num_cities=500, num_flights=10000, max_cost=1000)

Test 1:
Source: 63, Destination: 15, Max Stops: 376
Result: 253
Time Taken: 0.0033 seconds
----------------------------------------
Test 2:
Source: 8, Destination: 112, Max Stops: 314
Result: 552
Time Taken: 0.0078 seconds
----------------------------------------
Test 3:
Source: 206, Destination: 65, Max Stops: 147
Result: 454
Time Taken: 0.0047 seconds
----------------------------------------
Test 4:
Source: 83, Destination: 415, Max Stops: 131
Result: 246
Time Taken: 0.0029 seconds
----------------------------------------
Test 5:
Source: 168, Destination: 275, Max Stops: 249
Result: 313
Time Taken: 0.0040 seconds
----------------------------------------


In [54]:
# Testare l'algoritmo su dataset sintetici con prima implementazione::::
test_algorithm_with_synthetic_data(find_cheapest_route_v2, num_tests=5, num_cities=500, num_flights=10000, max_cost=1000)

Test 1:
Source: 130, Destination: 404, Max Stops: 103
Result: 260
Time Taken: 0.0023 seconds
----------------------------------------
Test 2:
Source: 399, Destination: 152, Max Stops: 399
Result: 398
Time Taken: 0.0214 seconds
----------------------------------------
Test 3:
Source: 26, Destination: 78, Max Stops: 360
Result: 292
Time Taken: 0.0015 seconds
----------------------------------------
Test 4:
Source: 220, Destination: 120, Max Stops: 121
Result: 400
Time Taken: 0.0111 seconds
----------------------------------------
Test 5:
Source: 252, Destination: 370, Max Stops: 199
Result: 313
Time Taken: 0.0044 seconds
----------------------------------------


##### *Here's a tesk with edge case , just to see how it works with exeptions*

In [55]:
def edge_case_tests(algorithm):
    """
    Test the algorithm with edge cases.
    
    Args:
        algorithm (function): The algorithm function to test.
    
    Returns:
        None
    """
    print("Running Edge Case Tests...\n")
    
    # Case 1: Disconnected Graph
    print("Test 1: Disconnected Graph")
    n = 4
    flights = [[0, 1, 100], [2, 3, 200]]  # No connection between src (0) and dst (3)
    src, dst, k = 0, 3, 1
    result = algorithm(n, flights, src, dst, k)
    print(f"Expected: -1, Got: {result}\n")
    
    # Case 2: Negative Weight Edge
    print("Test 2: Negative Weight Edge")
    n = 3
    flights = [[0, 1, 100], [1, 2, -50], [0, 2, 200]]  # Negative cost not supported by Dijkstra's algorithm
    src, dst, k = 0, 2, 1
    result = algorithm(n, flights, src, dst, k)
    print(f"Expected: Behavior undefined (algorithm assumes positive weights), Got: {result}\n")
    
    # Case 3: Missing Route
    print("Test 3: Missing Route")
    n = 3
    flights = []  # No flights available
    src, dst, k = 0, 2, 1
    result = algorithm(n, flights, src, dst, k)
    print(f"Expected: -1, Got: {result}\n")
    
    # Case 4: Single Node Graph
    print("Test 4: Single Node Graph")
    n = 1
    flights = []  # Only one city, no flights
    src, dst, k = 0, 0, 0
    result = algorithm(n, flights, src, dst, k)
    print(f"Expected: 0 (same city), Got: {result}\n")
    
    # Case 5: Exact Stop Limit
    print("Test 5: Exact Stop Limit")
    n = 4
    flights = [[0, 1, 100], [1, 2, 100], [2, 3, 100]]
    src, dst, k = 2, 3, 0  # Can only take one flight (exact stop limit)
    result = algorithm(n, flights, src, dst, k)
    print(f"Expected: 100, Got: {result}\n")

    if any(cost < 0 for _, _, cost in flights):
        raise ValueError("Negative weights detected. Algorithm supports only positive weights.")

In [56]:
# Eseguire i test per i casi limite
edge_case_tests(find_cheapest_route_v1)

Running Edge Case Tests...

Test 1: Disconnected Graph
Expected: -1, Got: -1

Test 2: Negative Weight Edge
Expected: Behavior undefined (algorithm assumes positive weights), Got: 50

Test 3: Missing Route
Expected: -1, Got: -1

Test 4: Single Node Graph
Expected: 0 (same city), Got: 0

Test 5: Exact Stop Limit
Expected: 100, Got: 100



In [57]:
def validate_and_run(algorithm, n, flights, src, dst, k):
    """
    Validate input and run the algorithm.
    
    Args:
        algorithm (function): The algorithm function.
        n (int): Number of cities.
        flights (list): List of flights.
        src (int): Source city.
        dst (int): Destination city.
        k (int): Maximum number of stops.
    
    Returns:
        int: Result of the algorithm or validation error.
    """
    # Check for negative weights
    if any(cost < 0 for _, _, cost in flights):
        print("Error: Negative weights detected. Algorithm supports only positive weights.")
        return None
    return algorithm(n, flights, src, dst, k)

In [58]:
# Test negative weights with validation
print(validate_and_run(find_cheapest_route_v1, 3, [[0, 1, 100], [1, 2, 50], [0, 2, 200]], 0, 2, 1))

150


### **Analysis of the algorithm's efficiency:**

#### **Time Complexity Analysis:**
1. Worst-case Time Complexity: $O(k * n^2)$
   - The queue sorting operation takes $O(n log n)$
   - Nested loop for exploring flights for each city
   - Potential exploration of multiple routes for each city-stop combination

#### **Space Complexity Analysis:**
1. Worst-case Space Complexity: $O(k * n)$
   - Adjacency list: $O(m)$, where m is number of flights
   - Distance dictionary:$O(k * n)$
   - Queue storage: $O(k * n)$

#### Efficiency Evaluation for Large Graphs $(n > 100)$:

Limitations:
1. Quadratic time complexity becomes problematic
   - For $n = 100$, time complexity becomes $O(10,000)$

Performance Bottlenecks:
- Repeated route explorations
- Sorting queue in each iteration
- Storing multiple route variations

Inefficiency Indicators:
- Becomes computationally expensive for dense graphs
- High memory consumption
- Slow for real-time routing applications

Optimization Strategies:
1. Use priority queue instead of list sorting
2. Implement more aggressive pruning
3. Consider alternative approaches like Dijkstra's with modifications

Recommended Improvements:
- Use heapq for efficient queue management
- Implement early termination conditions
- Reduce redundant route explorations

Practical Recommendation:
- Suitable for small to medium-sized graphs $(n < 50)$
- Not recommended for large, complex networks without significant optimization

Would you like me to elaborate on any specific aspect of the complexity analysis?

#### Let's dive deep into the efficiency for large graphs:

Efficiency Analysis for Large Graphs $(n > 100)$:

Computational Challenges:
1. Algorithmic Complexity
   - Current implementation: $O(k * n^2)$
   - For $n = 100$, computational steps become massive
   - Quadratic growth means exponential time increase

2. Memory Constraints
   - Stores multiple route variations
   - $O(k * n)$ space complexity becomes memory-intensive
   - Risk of memory overflow for large graphs

Performance Breakdown:
- Small Graphs $(n < 50)$: Very Efficient
- Medium Graphs $(50 < n < 100)$: Acceptable Performance
- Large Graphs $(n > 100)$: Significant Performance Degradation

Specific Inefficiencies:
- Repeated route calculations
- Extensive queue management
- Redundant path explorations
- High computational overhead

Practical Implications:
- Real-time routing becomes challenging
- Not suitable for large transportation networks
- Significant computational time and resource consumption

Mitigation Strategies:
1. Advanced Pruning Techniques
   - Implement more aggressive route filtering
   - Early termination of unpromising routes

2. Algorithmic Optimizations
   - Use priority queue (heapq)
   - Implement more efficient graph traversal
   - Reduce redundant computations

3. Alternative Approaches
   - Dijkstra's algorithm with modifications
   - A* search algorithm
   - Dynamic programming with memoization

Recommendation:
- Not recommended for graphs with n > 100 without significant optimization
- Requires substantial algorithmic redesign for large-scale applications

Comparative Efficiency:
- Current Algorithm:$O(k * n^2)$
- Desired Algorithm: $O(k * n * log(n))$ or better

Would you like me to elaborate on potential optimization techniques?

## **Optimized approach using Dijkstra's algorithm with a priority queue:**

In [59]:
import heapq

def find_cheapest_route_optimized(n, flights, src, dst, k):
    # Create adjacency list
    graph = [[] for _ in range(n)]
    for start, end, price in flights:
        graph[start].append((end, price))
    
    # Priority queue to store (cost, city, stops_remaining)
    priotiy_queue = [(0, src, k + 1)]
    
    # Track visited states to prevent redundant explorations
    visited = {}
    
    while priotiy_queue:
        # Extract the cheapest route
        current_cost, current_city, stops_left = heapq.heappop(priotiy_queue)
        
        # Destination reached
        if current_city == dst:
            return current_cost
        
        # Skip if this state has been seen with better conditions
        state_key = (current_city, stops_left)
        if state_key in visited and visited[state_key] <= current_cost:
            continue
        
        # Mark current state
        visited[state_key] = current_cost
        
        # No more stops available
        if stops_left == 0:
            continue
        
        # Explore neighboring cities
        for next_city, flight_cost in graph[current_city]:
            new_cost = current_cost + flight_cost
            
            # Add to priority queue
            heapq.heappush(priotiy_queue, (new_cost, next_city, stops_left - 1))
    
    # No route found
    return -1

# Optimization Analysis
# Time Complexity: O(k * E * log(V))
#   - E: Number of edges (flights)
#   - V: Number of vertices (cities)
#   - k: Maximum number of stops

# Space Complexity: O(V + E)

In [60]:
n = 4  
flights = [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]]  
src = 0  
dst = 3  
k = 1  

###

print(find_cheapest_route_v1(n, flights, src, dst, k))
print(find_cheapest_route_v2(n, flights, src, dst, k))
print(find_cheapest_route_optimized(n, flights, src, dst, k))

700
700
700


## Pseudocode for the optimied ALGORITHM FindCheapestRouteOptimized(n, flights, src, dst, k)

    // Create adjacency list representation of flights
    INITIALIZE graph as 2D list with n empty lists
    FOR EACH flight IN flights DO
        ADD (destination, cost) to graph[source]
    
    // Priority queue to manage routes
    INITIALIZE priorityQueue PQ
    PQ.push((0, src, k+1))  // (current_cost, current_city, remaining_stops)
    
    // Visited state tracking
    INITIALIZE visited_states as dictionary
    
    WHILE PQ is not empty DO
        (current_cost, current_city, stops_left) = PQ.pop()
        
        // Destination reached
        IF current_city == dst THEN
            RETURN current_cost
        
        // Avoid redundant explorations
        state_key = (current_city, stops_left)
        IF state_key exists in visited_states AND 
           visited_states[state_key] <= current_cost THEN
            CONTINUE
        
        // Update visited states
        visited_states[state_key] = current_cost
        
        // No more stops available
        IF stops_left == 0 THEN
            CONTINUE
        
        // Explore neighboring cities
        FOR EACH (next_city, flight_cost) IN graph[current_city] DO
            new_cost = current_cost + flight_cost
            PQ.push((new_cost, next_city, stops_left - 1))
    
    // No route found
    RETURN -1

# Time Complexity Analysis of Cheapest Route Algorithm

## Algorithmic Complexity Breakdown

### 1. Theoretical Complexity
- **Notation:** $O(E * log(V) * k)$
  - E: Number of Edges (Flights)
  - V: Number of Vertices (Cities)
  - k: Maximum Number of Stops

### 2. Complexity Components

| Component | Operation | Complexity | Explanation |
|-----------|-----------|------------|-------------|
| Graph Creation | Adjacency List | $O(E)$ | Iterate through all flights |
| Priority Queue | Insertion/Extraction | $O(log(V))$ | Heap operations |
| Route Exploration | Traversal | $O(k)$ | Limited by stop constraints |


### 3. Performance Scenarios

| Graph Size | Performance | Scalability |
|------------|-------------|-------------|
| Small $(n < 50)$ | ⭐⭐⭐⭐⭐ | Extremely Efficient |
| Medium $(50-100)$ | ⭐⭐⭐⭐ | Very Good |
| Large $(n > 100)$ | ⭐⭐⭐ | Good with Optimization |

### 4. Computational Breakdown


Time Complexity $O(E * log(V) * k)$ means:
- Each edge potentially explored
- Logarithmic selection of routes
- Constrained by maximum stops


### 5. Space vs Time Trade-off


#### Space Complexity: 
$O(V + E + k)$ 
$space_usage = cities + flights + max_stops$


### 6. Big O Notation Comparison

| Algorithm | Time Complexity | Efficiency |
|-----------|-----------------|------------|
| Initial Approach | $O(k * n²)$ | Poor Scaling |
| Optimized Approach | $O(E * log(V) * k)$ | Efficient Scaling |

### 7. Practical Implications

- ✅ Logarithmic route selection
- ✅ Constrained by stops
- ✅ Efficient for most routing scenarios
- ❗ Performance depends on graph density

### 8. Optimization Techniques

1. Priority Queue Management
2. Visited State Memoization
3. Early Route Termination
4. Intelligent Pruning

## LLM optimazied version of the algorithm :: 


In [61]:
import heapq
from collections import defaultdict

def find_cheapest_route_optimized_v3(n, flights, src, dst, k):
    # Create adjacency list
    graph = defaultdict(list)
    for start, end, price in flights:
        graph[start].append((end, price))
    
    # Priority queue to store (cost, city, stops)
    priority_queue = [(0, src, 0)]  # cost, current city, current stops
    
    # Dictionary to store the minimum cost to reach a city with a given number of stops
    min_cost_with_stops = defaultdict(lambda: float('inf'))
    min_cost_with_stops[(src, 0)] = 0
    
    while priority_queue:
        current_cost, current_city, current_stops = heapq.heappop(priority_queue)
        
        # Stop if we reach the destination
        if current_city == dst:
            return current_cost
        
        # If stops exceed k, skip processing this path
        if current_stops > k:
            continue
        
        # Explore neighbors
        for neighbor, flight_cost in graph[current_city]:
            new_cost = current_cost + flight_cost
            
            # Push to the queue only if this path is better
            if new_cost < min_cost_with_stops[(neighbor, current_stops + 1)]:
                min_cost_with_stops[(neighbor, current_stops + 1)] = new_cost
                heapq.heappush(priority_queue, (new_cost, neighbor, current_stops + 1))
    
    # If no route is found
    return -1

In [62]:
n = 4  
flights = [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]]  
src = 0  
dst = 3  
k = 1  

###

print(find_cheapest_route_optimized_v3(n, flights, src, dst, k))

700


## Comparison Table

| **Criteria**            | **Original Algorithm**                                         | **Optimized Algorithm (v2 - LLM one's)**                          |
|--------------------------|---------------------------------------------------------------|-------------------------------------------------------|
| **Data Structure**       | Standard adjacency list, `visited` dictionary                | `defaultdict` for adjacency list and tracking states |
| **Optimization**         | Checks redundant paths but may explore unnecessary states     | Uses `min_cost_with_stops` to prune unnecessary paths |
| **Performance**          | Slightly slower due to redundant states                      | Faster due to better pruning                          |
| **Time Complexity**      | $(O(k \cdot E \cdot \log(V))) $                               | $(O(k \cdot E \cdot \log(V))$) (same complexity, but fewer operations) |
| **Space Complexity**     | $(O(V + E))$                                            | $(O(V + E)$) (slightly more efficient dictionary usage) |
| **Correctness**          | Correct, but potentially less efficient for large graphs      | Correct and handles large graphs better               |